### Final Project - Data Hawks
---

#### Team Members:
> #### Sai Shishir Ailneni (U38559388)
> #### Sai Koushik Thatipamula (U18895638)
> #### Deepshikha Sherpally (U57605062)
> #### Akshitha Katta (U38519838)
> #### Rohith Sahini (U18270323)
---

### Notebook : Data Preprocessing
---

#### Notebook Description:
This Jupyter Notebook provides a step-by-step guide for preprocessing a dataset. Data preprocessing is a crucial step in preparing data for machine learning and analysis. In this notebook, we will cover the following preprocessing steps:

1. **Loading the Dataset**: We will start by loading the raw dataset to be processed.

2. **Exploratory Data Analysis (EDA)**: Before preprocessing, we'll perform some initial exploration to understand the dataset's structure and identify potential issues.

3. **Handling Missing Data**: We will address missing values in the dataset by either imputing them or removing rows/columns as appropriate.

4. **Feature Selection**: We will determine which features are relevant for our analysis and drop any unnecessary or redundant columns.

5. **Standardization**: We will scale numerical features to have a mean of 0 and a standard deviation of 1 (standardization). This step ensures that all numerical features have a similar scale.

6. **Dummy Encoding**: For categorical features, we will perform one-hot encoding (dummy encoding) to convert them into a numerical format suitable for machine learning algorithms.

---

#### Step-1 : Importing Libraries

> In this Step we Import all the required libraries for data prerprocessing

In [1]:
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

> Setting a seed as to get same results everytime

In [2]:
np.random.seed(42)

#### Step-2: Reading the data

> In this step we read the csv data into a pandas dataframe.

In [3]:
df = pd.read_csv("diabetic_data.csv")

C:\Users\navee\AppData\Local\Temp\ipykernel_11248\1493456993.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("diabetic_data.csv")


In [4]:
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


#### Step-3: Exploratory Data Analytics (EDA)

>In this step we'll perform some initial exploration to understand the dataset's structure and identify potential issues.

In [5]:
df.shape

(101766, 50)

We have 101,766 observations and 50 Features

---

##### 3.1 - Checking for a null values in the features

In [6]:
df.isna().sum()

encounter_id                    0
patient_nbr                     0
race                         2273
gender                          0
age                             0
weight                      98569
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  40256
medical_specialty           49949
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                         21
diag_2                        358
diag_3                       1423
number_diagnoses                0
max_glu_serum                   0
A1Cresult                       0
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide 

We see that there are some significant number of null values in some features, Weight, Payer_code and medical_speciality.

---

##### 3.2 Checking for data types and info of features

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      99493 non-null   object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    3197 non-null    object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                61510 non-null   object
 11  medical_specialty         51817 non-null   object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

As we observe most of the features in the data are objects(Categorical), and there are very few numerical variables.

---

##### 3.3 Feature Engineering

> We check for the distribution of the target variable

In [8]:
df["readmitted"].value_counts()

NO     54864
>30    35545
<30    11357
Name: readmitted, dtype: int64

- we impute the our target variable in the following way.
> '<30' as 1 and anything else as 0
- This is because we would like to predict the early readmission rate and so we consider '>30' and 'NO' as not "early readmission"

In [9]:
def map_readmitted(value):
    if value == '<30':
        return 1
    else:
        return 0

In [10]:
df["readmitted"] = df["readmitted"].apply(map_readmitted)

In [11]:
df["readmitted"].value_counts()

0    90409
1    11357
Name: readmitted, dtype: int64

> As we see there is a lot of imbalance in the data.

In [12]:
df.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

In [13]:
pd.set_option("display.max_rows", None)

- In the dataset, we have both encounter_id and patient_nbr. patient_nbr is a unique id given to each patient where as encounter_id is an id generated everytime a patient visits the hospital.

- So there might be duplicate values in patient_nbr as each person might visit the hospital multiple times. 

- Hence we remove the duplicate rows for the same patient.

In [14]:
df = df.drop_duplicates(subset=['patient_nbr'])

In [15]:
df.shape

(71518, 50)

> After removing the duplicates from patient_nbr we see that there are 71,518 unique patient data available.

---

- As seen the section 3.1 there are a lot of null values in these columns, hence we drop them.

In [16]:
columns_to_drop = ['weight','payer_code','medical_specialty']

In [17]:
# Weight, payer_code and medical_speciality has more than 50% null values so we drop them

df = df.drop(columns_to_drop,axis = 1)

In [18]:
df.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses',
       'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton',
       'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

---

**Feature**: age

In [19]:
df['age'].value_counts()

[70-80)     18210
[60-70)     15960
[50-60)     12466
[80-90)     11589
[40-50)      6878
[30-40)      2699
[90-100)     1900
[20-30)      1127
[10-20)       535
[0-10)        154
Name: age, dtype: int64

- For age we create it as an ordinal feature
- We Impute the age in the following way.
- (0-10) & (10-20) as 0
- (20-30) & (30-40) as 1
- (40-50) & (50-60) as 2
- (60-70) & (70-80) as 3
- (80-90) & (90-100) as 4

In [20]:
def map_age_to_numeric(age_group):
    if age_group in ('[0-10)', '[10-20)'):
        return 0
    elif age_group in ('[20-30)', '[30-40)'):
        return 1
    elif age_group in ('[40-50)', '[50-60)'):
        return 2
    elif age_group in ('[60-70)', '[70-80)'):
        return 3
    elif age_group in ('[80-90)', '[90-100)'):
        return 4
    else:
        return age_group

In [21]:
df['age'] = df['age'].apply(map_age_to_numeric)

In [22]:
df['age'].value_counts()

3    34170
2    19344
4    13489
1     3826
0      689
Name: age, dtype: int64

---

**Feature**: admission_type_id

In [23]:
df['admission_type_id'].value_counts()

1    36490
3    13917
2    13028
6     4588
5     3174
8      291
7       21
4        9
Name: admission_type_id, dtype: int64

- admission type id is Integer identifier corresponding to 9 distinct values, for example, emergency, urgent, elective, newborn, and not available.
- We impute the admission type id in the following way
- 1 or 2 as Emergency
- 3 as Elective
- 4 as New Born
- 7 as Trauma Center
- anything else as other
- The source for the above information is : https://www.cms.gov/regulations-and-guidance/guidance/transmittals/downloads/r1775cp.pdf

In [24]:
def map_admission_type(admission_type_id):
    if admission_type_id == 1.0 or admission_type_id == 2.0:
        return "Emergency"
    elif admission_type_id == 3.0:
        return "Elective"
    elif admission_type_id == 4.0:
        return "New_Born"
    elif admission_type_id == 7.0:
        return "Trauma_Center"
    else:
        return "Other"

In [25]:
df['admission_type_id'] = df['admission_type_id'].apply(map_admission_type)

In [26]:
df['admission_type_id'].value_counts()

Emergency        49518
Elective         13917
Other             8053
Trauma_Center       21
New_Born             9
Name: admission_type_id, dtype: int64

> Since there are very less values for both trauma center and new born we drop the rows which have these values.

In [27]:
values_to_drop = ['Trauma_Center', 'New_Born']

In [28]:
df = df[~df['admission_type_id'].isin(values_to_drop)]

In [29]:
df['admission_type_id'].value_counts()

Emergency    49518
Elective     13917
Other         8053
Name: admission_type_id, dtype: int64

---

**Feature** : discharge_disposition_id

In [30]:
df['discharge_disposition_id'].value_counts()

1     44307
3      8778
6      8282
18     2474
2      1538
22     1407
11     1074
5       913
25      778
4       541
7       409
23      260
13      243
14      218
28       90
8        73
15       40
24       25
9         9
17        8
10        6
19        6
27        3
16        3
12        2
20        1
Name: discharge_disposition_id, dtype: int64

- Discharge disposition id is Integer identifier corresponding to 29 distinct values, for example, discharged to home, expired, and not available
- So we Impute the values into three ways, "Discharged to home" , "other" or we put it as null value so we can impute it using SimpleImputer().
- Source for the information is https://med.noridianmedicare.com/web/jfa/topics/claim-submission/patient-discharge-status-codes

In [31]:
def map_discharge_disposition_id(discharge_id):
    if discharge_id in [1, 6, 8, 13, 19]:
        return "Discharged to Home"
    elif discharge_id in [2, 3, 4, 5, 7, 9, 10, 11, 12, 14, 15, 16, 17, 20, 21, 22, 23, 24, 27, 28, 29]:
        return "Other"
    else:
        return np.nan

In [32]:
df['discharge_disposition_id'] = df['discharge_disposition_id'].apply(map_discharge_disposition_id)

In [33]:
df['discharge_disposition_id'].value_counts()

Discharged to Home    52911
Other                 15325
Name: discharge_disposition_id, dtype: int64

---

- **Feature** : admission_source_id

In [34]:
df['admission_source_id'].value_counts()

7     38267
1     22005
17     4949
4      2579
6      1801
2       926
5       545
20      155
3       138
9        95
8        12
10        7
22        4
25        2
14        1
11        1
13        1
Name: admission_source_id, dtype: int64

- admission_source_id is Integer identifier corresponding to 21 distinct values, for example, physician referral, emergency room, and transfer from a hospital.
- So, we impute the values into 4 ways Referral, Emergency, Other or Null for which we can use SimpleImputer() to later impute the value.
- Source for the information related to codes is : https://www.cms.gov/regulations-and-guidance/guidance/transmittals/downloads/r1775cp.pdf

In [35]:
def map_admission_source(admission_source_id):
    if admission_source_id in [1, 2, 3]:
        return "Referral"
    elif admission_source_id == 7:
        return "Emergency"
    elif admission_source_id in [4, 5, 6, 8, 9, 10, 11, 13, 14, 22, 25]:
        return "Other"
    else:
        return np.nan

In [36]:
df['admission_source_id'] = df['admission_source_id'].apply(map_admission_source)

In [37]:
df['admission_source_id'].value_counts()

Emergency    38267
Referral     23069
Other         5048
Name: admission_source_id, dtype: int64

---

> As mentioned above we impute the NULL values with the most frequent value(mode) as stated above.

In [38]:
df['discharge_disposition_id'] = df['discharge_disposition_id'].fillna(df['discharge_disposition_id'].mode()[0])

df['admission_source_id'] = df['admission_source_id'].fillna(df['admission_source_id'].mode()[0])

In [39]:
df['admission_source_id'].value_counts()

Emergency    43371
Referral     23069
Other         5048
Name: admission_source_id, dtype: int64

In [40]:
df['discharge_disposition_id'].value_counts()

Discharged to Home    56163
Other                 15325
Name: discharge_disposition_id, dtype: int64

---

- **Feature** : race

> As we have seen in Section 3.2 there are some missing values in race, so we impute them using mode.

In [41]:
df['race'] = df['race'].fillna(df['race'].mode()[0])

In [42]:
df['race'].value_counts()

Caucasian          55416
AfricanAmerican    12883
Hispanic            1515
Other               1178
Asian                496
Name: race, dtype: int64

- As in case of race we impute Hispanic and Asian as well into other as well.

In [43]:
categories_to_impute = ['Hispanic', 'Asian']

df['race'] = df['race'].replace(categories_to_impute, 'Other')

In [44]:
df['race'].value_counts()

Caucasian          55416
AfricanAmerican    12883
Other               3189
Name: race, dtype: int64

---

- **Feature** : number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3

In [45]:
df.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses',
       'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton',
       'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

> We drop these features, as we belive that in this context this information is redundant and is already provided by other features in the data.

In [46]:
columns_todrop = ['number_outpatient','number_emergency','number_inpatient','diag_1','diag_2','diag_3']

In [47]:
df = df.drop(columns_todrop,axis = 1)

In [48]:
df.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

In [49]:
df.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,0,Other,Discharged to Home,Referral,1,41,...,No,No,No,No,No,No,No,No,No,0
1,149190,55629189,Caucasian,Female,0,Emergency,Discharged to Home,Emergency,3,59,...,No,Up,No,No,No,No,No,Ch,Yes,0
2,64410,86047875,AfricanAmerican,Female,1,Emergency,Discharged to Home,Emergency,2,11,...,No,No,No,No,No,No,No,No,Yes,0
3,500364,82442376,Caucasian,Male,1,Emergency,Discharged to Home,Emergency,2,44,...,No,Up,No,No,No,No,No,Ch,Yes,0
4,16680,42519267,Caucasian,Male,2,Emergency,Discharged to Home,Emergency,1,51,...,No,Steady,No,No,No,No,No,Ch,Yes,0


---

- **Feature**: Medicines and Tests

In [50]:
medicine_list = ['max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton',
       'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed']

In [51]:
for i in medicine_list:
    print("*****************",i,"*****************")
    print(df[i].value_counts())

***************** max_glu_serum *****************
None    68032
Norm     1731
>200      969
>300      756
Name: max_glu_serum, dtype: int64
***************** A1Cresult *****************
None    58508
>8       6304
Norm     3787
>7       2889
Name: A1Cresult, dtype: int64
***************** metformin *****************
No        56504
Steady    13711
Up          838
Down        435
Name: metformin, dtype: int64
***************** repaglinide *****************
No        70562
Steady      824
Up           73
Down         29
Name: repaglinide, dtype: int64
***************** nateglinide *****************
No        70990
Steady      474
Up           16
Down          8
Name: nateglinide, dtype: int64
***************** chlorpropamide *****************
No        71416
Steady       67
Up            4
Down          1
Name: chlorpropamide, dtype: int64
***************** glimepiride *****************
No        67745
Steady     3370
Up          235
Down        138
Name: glimepiride, dtype: int64
******

> We drop all the variables that have more than 90% of data as a single value.

In [52]:
medicine_to_drop = ['chlorpropamide','acetohexamide', 'tolbutamide','miglitol', 'troglitazone','tolazamide','examide','citoglipton', 'glipizide-metformin','glimepiride-pioglitazone','metformin-rosiglitazone','metformin-pioglitazone']

In [53]:
df = df.drop(medicine_to_drop,axis = 1)

In [54]:
df.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,glipizide,glyburide,pioglitazone,rosiglitazone,acarbose,insulin,glyburide-metformin,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,0,Other,Discharged to Home,Referral,1,41,...,No,No,No,No,No,No,No,No,No,0
1,149190,55629189,Caucasian,Female,0,Emergency,Discharged to Home,Emergency,3,59,...,No,No,No,No,No,Up,No,Ch,Yes,0
2,64410,86047875,AfricanAmerican,Female,1,Emergency,Discharged to Home,Emergency,2,11,...,Steady,No,No,No,No,No,No,No,Yes,0
3,500364,82442376,Caucasian,Male,1,Emergency,Discharged to Home,Emergency,2,44,...,No,No,No,No,No,Up,No,Ch,Yes,0
4,16680,42519267,Caucasian,Male,2,Emergency,Discharged to Home,Emergency,1,51,...,Steady,No,No,No,No,Steady,No,Ch,Yes,0


- For the values in tests and medicines we impute them as 0 and 1. If it is No we impute it as 0 and if it is anything else we impute them as 1.
- For most medicines we have Either No or Steady, so we impute them in the above mentioned way.

In [55]:
medicine_to_impute = ['max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide','glimepiride','glipizide','glyburide','pioglitazone', 'rosiglitazone', 'acarbose','insulin','glyburide-metformin','change', 'diabetesMed']

In [56]:
def map_medicine_values(value):
    if value == 'No':
        return 0
    else:
        return 1

In [57]:
for i in medicine_to_impute:
    df[i] = df[i].apply(map_medicine_values)

In [58]:
df.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,glipizide,glyburide,pioglitazone,rosiglitazone,acarbose,insulin,glyburide-metformin,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,0,Other,Discharged to Home,Referral,1,41,...,0,0,0,0,0,0,0,0,0,0
1,149190,55629189,Caucasian,Female,0,Emergency,Discharged to Home,Emergency,3,59,...,0,0,0,0,0,1,0,1,1,0
2,64410,86047875,AfricanAmerican,Female,1,Emergency,Discharged to Home,Emergency,2,11,...,1,0,0,0,0,0,0,0,1,0
3,500364,82442376,Caucasian,Male,1,Emergency,Discharged to Home,Emergency,2,44,...,0,0,0,0,0,1,0,1,1,0
4,16680,42519267,Caucasian,Male,2,Emergency,Discharged to Home,Emergency,1,51,...,1,0,0,0,0,1,0,1,1,0


---

In [59]:
df = df.drop(['encounter_id','patient_nbr'],axis = 1)

In [60]:
df.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,...,glipizide,glyburide,pioglitazone,rosiglitazone,acarbose,insulin,glyburide-metformin,change,diabetesMed,readmitted
0,Caucasian,Female,0,Other,Discharged to Home,Referral,1,41,0,1,...,0,0,0,0,0,0,0,0,0,0
1,Caucasian,Female,0,Emergency,Discharged to Home,Emergency,3,59,0,18,...,0,0,0,0,0,1,0,1,1,0
2,AfricanAmerican,Female,1,Emergency,Discharged to Home,Emergency,2,11,5,13,...,1,0,0,0,0,0,0,0,1,0
3,Caucasian,Male,1,Emergency,Discharged to Home,Emergency,2,44,1,16,...,0,0,0,0,0,1,0,1,1,0
4,Caucasian,Male,2,Emergency,Discharged to Home,Emergency,1,51,0,8,...,1,0,0,0,0,1,0,1,1,0


In [61]:
df.columns

Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin',
       'repaglinide', 'nateglinide', 'glimepiride', 'glipizide', 'glyburide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'insulin',
       'glyburide-metformin', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

#### Step-4 : Creating Dummy Variables

> In This step we create dummy variables for the categorical variables.

In [62]:
cat_columns = ['race','gender','admission_type_id','discharge_disposition_id', 'admission_source_id']

In [63]:
df = pd.get_dummies(df, prefix_sep='_', dummy_na=False, drop_first=True, columns=cat_columns)

In [64]:
df.head()

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,...,readmitted,race_Caucasian,race_Other,gender_Male,gender_Unknown/Invalid,admission_type_id_Emergency,admission_type_id_Other,discharge_disposition_id_Other,admission_source_id_Other,admission_source_id_Referral
0,0,1,41,0,1,1,1,1,0,0,...,0,1,0,0,0,0,1,0,0,1
1,0,3,59,0,18,9,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
2,1,2,11,5,13,6,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,1,2,44,1,16,7,1,1,0,0,...,0,1,0,1,0,1,0,0,0,0
4,2,1,51,0,8,5,1,1,0,0,...,0,1,0,1,0,1,0,0,0,0


---

#### Step-5 : Train-Test Split

> We split the data into 80-20 as we have a lot of imbalance in the data, we would be doing undersampling, so as to have enough data for train we take a 80 split for the train data and 20 split for the test data

In [65]:
# random_state is set to a defined value to get the same partitions when re-running the code
train_data= df.sample(frac=0.8, random_state=1)
# assign rows that are not already in the training set, into validation 
valid_data = df.drop(train_data.index)

print('Training   : ', train_data.shape)
print('Validation : ', valid_data.shape)

Training   :  (57190, 31)
Validation :  (14298, 31)


In [66]:
train_data.head()

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,...,readmitted,race_Caucasian,race_Other,gender_Male,gender_Unknown/Invalid,admission_type_id_Emergency,admission_type_id_Other,discharge_disposition_id_Other,admission_source_id_Other,admission_source_id_Referral
44748,2,5,38,0,16,9,1,1,1,0,...,0,0,0,1,0,1,0,0,0,0
11187,2,2,34,3,8,6,1,1,0,0,...,0,0,0,1,0,1,0,0,0,0
45442,3,3,2,1,6,7,1,1,1,0,...,0,0,1,0,0,1,0,0,0,0
31492,2,6,31,0,17,5,1,1,1,0,...,0,1,0,0,0,1,0,0,1,0
96962,3,6,69,1,32,9,1,1,0,0,...,0,1,0,1,0,1,0,0,0,0


---

#### Step-6 : Data Standardization

In [67]:
columns_to_standardize = ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_diagnoses']

In [68]:
df[columns_to_standardize].describe()

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_diagnoses
count,71488.000000,71488.000000,71488.000000,71488.000000,71488.000000
mean,4.289097,43.073425,1.430156,15.705223,7.245496
std,2.949262,19.953728,1.759583,8.311210,1.994774
min,1.000000,1.000000,0.000000,1.000000,1.000000
25%,2.000000,31.000000,0.000000,10.000000,6.000000
50%,3.000000,44.000000,1.000000,14.000000,8.000000
75%,6.000000,57.000000,2.000000,20.000000,9.000000
max,14.000000,132.000000,6.000000,81.000000,16.000000


>As we see above, all the numerical data is in a different scale, hence we use standard scaler to standardize the data and bring them all into the same scale.

In [69]:
scaler = StandardScaler()

train_data[columns_to_standardize] = scaler.fit_transform(train_data[columns_to_standardize])
valid_data[columns_to_standardize] = scaler.fit_transform(valid_data[columns_to_standardize])

In [70]:
train_data.head()

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,...,readmitted,race_Caucasian,race_Other,gender_Male,gender_Unknown/Invalid,admission_type_id_Emergency,admission_type_id_Other,discharge_disposition_id_Other,admission_source_id_Other,admission_source_id_Referral
44748,2,0.241653,-0.255799,-0.812610,0.037538,0.879498,1,1,1,0,...,0,0,0,1,0,1,0,0,0,0
11187,2,-0.776204,-0.456624,0.898206,-0.925904,-0.622966,1,1,0,0,...,0,0,0,1,0,1,0,0,0,0
45442,3,-0.436918,-2.063225,-0.242338,-1.166765,-0.122145,1,1,1,0,...,0,0,1,0,0,1,0,0,0,0
31492,2,0.580938,-0.607243,-0.812610,0.157968,-1.123788,1,1,1,0,...,0,1,0,0,0,1,0,0,1,0
96962,3,0.580938,1.300595,-0.242338,1.964422,0.879498,1,1,0,0,...,0,1,0,1,0,1,0,0,0,0


In [71]:
valid_data.head()

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,...,readmitted,race_Caucasian,race_Other,gender_Male,gender_Unknown/Invalid,admission_type_id_Emergency,admission_type_id_Other,discharge_disposition_id_Other,admission_source_id_Other,admission_source_id_Referral
1,0,-0.437801,0.796822,-0.813642,0.267002,0.879806,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
9,4,2.606131,-0.496984,0.868628,0.267002,0.376507,1,1,0,0,...,0,1,0,0,0,0,0,1,1,0
15,3,2.606131,1.593010,1.990141,-0.332439,0.879806,1,1,0,0,...,0,0,0,1,0,1,0,1,0,0
18,3,0.238628,-0.397461,1.990141,0.866443,0.879806,1,1,0,0,...,0,0,0,1,0,1,0,0,0,0
27,2,-0.776015,-0.895078,0.307871,-0.572215,-2.139991,1,1,0,0,...,0,1,0,0,0,1,0,0,0,1


---

#### Step-7: Saving the train and test data

In [72]:
train_data.to_csv('train_data.csv')

In [73]:
valid_data.to_csv('valid_data.csv')